# Web Scrping NBA Players

In [116]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
url='http://www.nba.com/players'
htmldata=requests.get(url)

In [3]:
soup=BeautifulSoup(htmldata.content,'html5lib')

In [4]:
from selenium import webdriver

In [5]:
driver = webdriver.Firefox()
driver.get(url)
htmlSource = driver.page_source

In [6]:
soup=BeautifulSoup(htmlSource,'html5lib')

## Getting Player Names

In [7]:
playerList=soup.find_all('a',attrs={'class':'row playerList'})

In [8]:
playerName=[]
for names in playerList:
    name=names.get('title')
    playerName.append(name)
nbaplayers=pd.DataFrame(playerName,columns=['Name'])

In [9]:
nbaplayers.head(1)

,Name
0,Jaylen Adams


In [10]:
profileLink=[]
for links in playerList:
    link=links.get('href')
    profileLink.append('http://www.nba.com'+link)
nbaplayers['Profile_Link']=profileLink



In [11]:
nbaplayers.head(1)

,Name,Profile_Link
0,Jaylen Adams,http://www.nba.com/players/jaylen/adams/1629121


In [12]:
playerTeam=[]
playerTeamAbbr=[]
for i in range(len(playerList)):
    teamName=playerList[i].find('abbr').get('title')
    teamTitle=playerList[i].find('abbr').text
    playerTeam.append(teamName)
    playerTeamAbbr.append(teamTitle)
nbaplayers['Team']=playerTeam
nbaplayers['Abbr']=playerTeamAbbr

In [13]:
nbaplayers.head(1)

,Name,Profile_Link,Team,Abbr
0,Jaylen Adams,http://www.nba.com/players/jaylen/adams/1629121,Atlanta,ATL


In [14]:
photoLink=[]
for i in range(len(playerList)):
    path=playerList[i].find('img').get('data-src')
    photoLink.append('http:'+path)
nbaplayers['Photo']=photoLink


In [97]:
personalprofileLink=nbaplayers.Profile_Link[238]
c=requests.get(personalprofileLink)
soup1=BeautifulSoup(c.content,'html5lib')

In [114]:
"Vitals Details"

a=soup1.find_all('p',attrs={'class':'nba-player-vitals__top-info-imperial'})
b=soup1.find_all('p',attrs={'class':'nba-player-vitals__top-info-metric'})
print(a[0].text,a[1].text)
print(b[0].text.replace("/"," ").replace(" ",""),b[1].text.replace("/"," ").replace(" ",""))

6 ft8 in 240 lbs

2.03m 
108.9kg
2


In [57]:
c=soup1.find_all('span','nba-player-vitals__bottom-heading')
d=soup1.find_all('span','nba-player-vitals__bottom-info')
details=[]
for i in range(6):
    details.append((c[i].text.replace("\n"," ").replace(" ",""),d[i].text.replace("\n"," ").replace(" ","")))
details

[('BORN', '02/20/1987'),
 ('AGE', '32years'),
 ('FROM', 'WakeForest'),
 ('NBADEBUT', '2009'),
 ('YEARSINNBA', '9'),
 ('PREVIOUSLY',
  'MIA2016-19TOR2014-16MEM2013-14SAC2012-13TOR2010-12CHI2009-11')]

In [130]:
len(nbaplayers)

490

## Fetching Details of All Players.

In [158]:
p_details=pd.DataFrame(playerDetails,columns=['Height1','Height2','Weight1','Weight2','D.O.B','Age','Province','Debut','Years_in_Nba','Previous_Team'])
for i in nbaplayers.Profile_Link:
    url=i
    c=requests.get(url)
    soup1=BeautifulSoup(c.content,'html5lib')
    a=soup1.find_all('p','nba-player-vitals__top-info-imperial')
    b=soup1.find_all('p','nba-player-vitals__top-info-metric')
    d=soup1.find_all('span','nba-player-vitals__bottom-info')
    p_details=p_details.append({
        'Height1':a[0].text,
        'Height2':b[0].text.replace("/"," ").replace(" ","").replace("\n",""),
        'Weight1':a[1].text,
        'Weight2':b[1].text.replace("/"," ").replace(" ","").replace("\n"," "),
        'D.O.B':d[0].text.replace("\n"," ").replace(" ",""),
        'Age':d[1].text.replace("\n"," ").replace(" ",""),
        'Province':d[2].text.replace("\n"," ").replace(" ",""),
        'Debut':d[3].text.replace("\n"," ").replace(" ",""),
        'Years_in_Nba':d[4].text.replace("\n"," ").replace(" ",""),
        'Previous_Team':d[5].text.replace("\n"," ").replace(" ",""),
    },ignore_index=True)
    

In [184]:
nbaplayers=pd.concat([nbaplayers,p_details],axis=1)

In [185]:
nbaplayers.head()

,Name,Profile_Link,Team,Abbr,Photo,Height1,Height2,Weight1,Weight2,D.O.B,Age,Province,Debut,Years_in_Nba,Previous_Team
0,Jaylen Adams,http://www.nba.com/players/jaylen/adams/1629121,Atlanta,ATL,http://ak-static.cms.nba.com/wp-content/upload...,6 ft2 in,1.88m,190 lbs,86.2kg,05/04/1996,22years,St.Bonaventure,2018,0,ATL2018-19
1,Steven Adams,http://www.nba.com/players/steven/adams/203500,Oklahoma City,OKC,http://ak-static.cms.nba.com/wp-content/upload...,7 ft0 in,2.13m,265 lbs,120.2kg,07/20/1993,25years,Pittsburgh,2013,5,OKC2013-19
2,Bam Adebayo,http://www.nba.com/players/bam/adebayo/1628389,Miami,MIA,http://ak-static.cms.nba.com/wp-content/upload...,6 ft10 in,2.08m,255 lbs,115.7kg,07/18/1997,21years,Kentucky,2017,1,MIA2017-19
3,Deng Adel,http://www.nba.com/players/deng/adel/1629061,Cleveland,CLE,http://ak-static.cms.nba.com/wp-content/upload...,6 ft7 in,2.01m,200 lbs,90.7kg,02/01/1997,22years,—,2018,0,CLE2018-19
4,LaMarcus Aldridge,http://www.nba.com/players/lamarcus/aldridge/2...,San Antonio,SAS,http://ak-static.cms.nba.com/wp-content/upload...,6 ft11 in,2.11m,260 lbs,117.9kg,07/19/1985,33years,Texas,2006,12,SAS2015-19POR2006-15


In [188]:
nbaplayers.Photo[0]

'http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1629121.png'

In [211]:
from IPython.display import Image, HTML
paths=[]
for link in nbaplayers.Photo:
    path=link
    paths.append('<img src="'+path+'"/>')
    
df = pd.DataFrame(paths, columns = ['Image'])


temp=pd.concat([df,nbaplayers],axis=1)
HTML(temp.to_html(escape=False))


,Image,Name,Profile_Link,Team,Abbr,Photo,Height1,Height2,Weight1,Weight2,D.O.B,Age,Province,Debut,Years_in_Nba,Previous_Team
0,,Jaylen Adams,http://www.nba.com/players/jaylen/adams/1629121,Atlanta,ATL,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1629121.png,6 ft2 in,1.88m,190 lbs,86.2kg,05/04/1996,22years,St.Bonaventure,2018,0,ATL2018-19
1,,Steven Adams,http://www.nba.com/players/steven/adams/203500,Oklahoma City,OKC,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/203500.png,7 ft0 in,2.13m,265 lbs,120.2kg,07/20/1993,25years,Pittsburgh,2013,5,OKC2013-19
2,,Bam Adebayo,http://www.nba.com/players/bam/adebayo/1628389,Miami,MIA,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1628389.png,6 ft10 in,2.08m,255 lbs,115.7kg,07/18/1997,21years,Kentucky,2017,1,MIA2017-19
3,,Deng Adel,http://www.nba.com/players/deng/adel/1629061,Cleveland,CLE,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1629061.png,6 ft7 in,2.01m,200 lbs,90.7kg,02/01/1997,22years,—,2018,0,CLE2018-19
4,,LaMarcus Aldridge,http://www.nba.com/players/lamarcus/aldridge/200746,San Antonio,SAS,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/200746.png,6 ft11 in,2.11m,260 lbs,117.9kg,07/19/1985,33years,Texas,2006,12,SAS2015-19POR2006-15
5,,Rawle Alkins,http://www.nba.com/players/rawle/alkins/1628959,Chicago,CHI,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1628959.png,6 ft5 in,1.96m,225 lbs,102.1kg,10/29/1997,21years,—,2018,0,CHI2018-19
6,,Grayson Allen,http://www.nba.com/players/grayson/allen/1628960,Utah,UTA,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1628960.png,6 ft5 in,1.96m,198 lbs,89.8kg,10/08/1995,23years,Duke,2018,0,UTA2018-19
7,,Jarrett Allen,http://www.nba.com/players/jarrett/allen/1628386,Brooklyn,BKN,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1628386.png,6 ft11 in,2.11m,237 lbs,107.5kg,04/21/1998,20years,Texas,2017,1,BKN2017-19
8,,Kadeem Allen,http://www.nba.com/players/kadeem/allen/1628443,New York,NYK,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/1628443.png,6 ft3 in,1.9m,200 lbs,90.7kg,01/15/1993,26years,Arizona,2017,1,NYK2018-19BOS2017-18
9,,Al-Farouq Aminu,http://www.nba.com/players/al-farouq/aminu/202329,Portland,POR,http://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/202329.png,6 ft9 in,2.06m,220 lbs,99.8kg,09/21/1990,28years,WakeForest,2010,8,POR2015-19DAL2014-15NOP2011-14LAC2010-11
